<div style="font-family: 'Courier New', Courier, monospace; font-size: 1.8em; color: #f6f8fa; background-color: #24292e; border: 2px solid #444d56; border-radius: 6px; padding: 10px 20px; margin-top: 20px; box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);">
    <span style="font-weight: 600; font-size: 1.1em; letter-spacing: 0.5px; color: #f0f6fc;">1. Notebook Overview</span>
</div>


<div style="font-family: 'Arial', sans-serif; line-height: 1.7; color: #333; max-width: 1000px; margin: auto; padding: 15px;">

  <p style="font-size: 1.2em; margin-bottom: 20px;">
    Welcome to the final notebook in our three-part series! In this section, we will learn about the MicroOpt algorithm, which can be used to optimize the resource allocation for a given slice while maintaining the desired QoS.
  </p>

  <div align="center" style="margin: 25px 0; padding: 15px; background-color: white; border: 1px solid #444; border-radius: 8px;">
    <img src="images/microopt.png" alt="MicroOpt Overview" width="400"" />
  </div>

  <div align="center" style="margin: 25px 0; padding: 15px; background-color: white; border: 1px solid #444; border-radius: 8px;">
    <img src="images/algorithm.png" alt="MicroOpt Flowchart" width="400"" />
  </div>

  <p style="font-size: 1.2em; margin-bottom: 20px;">
    Before we begin, let's take a look at the `MicroOpt` class in <a href="https://github.com/sulaimanalmani/5GDynamicResourceAllocation/blob/main/resource_allocation.py" target="_blank" style="color: #2196F3; text-decoration: none;">resource_allocation.py</a> and identify the components shown in the figure above.
  </p>

</div>


<div style="font-family: 'Courier New', Courier, monospace; font-size: 1.8em; color: #f6f8fa; background-color: #24292e; border: 2px solid #444d56; border-radius: 6px; padding: 10px 20px; margin-top: 20px; box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);">
    <span style="font-weight: 600; font-size: 1.1em; letter-spacing: 0.5px; color: #f0f6fc;">2. Importing the necessary libraries</span>
</div>


In [15]:
%%capture
%load_ext autoreload
%autoreload 2

from data_generator import DataGenerator as DataGenerator
from matplotlib import pyplot as plt
import pandas as pd
import ipywidgets as widgets
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'white'
from IPython.display import display

from vnf_model import *
from slice_model import *
from resource_allocation import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

<div style="font-family: 'Courier New', Courier, monospace; font-size: 1.8em; color: #f6f8fa; background-color: #24292e; border: 2px solid #444d56; border-radius: 6px; padding: 10px 20px; margin-top: 20px; box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);">
    <span style="font-weight: 600; font-size: 1.1em; letter-spacing: 0.5px; color: #f0f6fc;">3. Resource Allocation using MicroOpt</span>
</div>


<div style="display: inline-block;">
    <h2 style="font-family: 'Courier New', Courier, monospace; font-size: 1.5em; color: #f0f6fc; background-color: #2d333b; border-left: 4px solid #3F51B5; padding: 8px 15px; border-radius: 4px; margin-top: 20px; box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);">
        3a. Loading the Slice Model
    </h2>
</div>


In [52]:
ran_data_gen = DataGenerator('./net_model_dataset/ran/input_dataset.pkl', './net_model_dataset/ran/output_dataset.pkl', vnf_type='RAN', norm_type='minmax')
ran_model = VNF_Model('ran', 6, [64, 32, 16], 5)
ran_model.to(device)
ran_model.load_weights('./data/saved_weights/ran/model.pth')

ovs_data_gen = DataGenerator('./net_model_dataset/ovs/input_dataset.pkl', './net_model_dataset/ovs/output_dataset.pkl', vnf_type='OvS', norm_type='minmax')
ovs_model = VNF_Model('ovs', 6, [64, 32, 16], 5)
ovs_model.to(device)
ovs_model.load_weights('./data/saved_weights/ovs/model.pth')

upf_data_gen = DataGenerator('./net_model_dataset/upf/input_dataset.pkl', './net_model_dataset/upf/output_dataset.pkl', vnf_type='UPF', norm_type='minmax')
upf_model = VNF_Model('upf', 6, [32, 16], 5)
upf_model.to(device)
upf_model.load_weights('./data/saved_weights/upf/model.pth')

data_gens = [upf_data_gen, ovs_data_gen, ran_data_gen]
vnf_models = [upf_model, ovs_model, ran_model]
slice_model = SliceModel(vnf_models, data_gens)


In [ ]:
resource_allocation = {'UPF': 200, #CPU (millicores)
                       'OVS': 50, #Throughput (Mbps)
                       'RAN': 1000} #CPU (millicores)
input_throughput = 35

output_throughput = slice_model.predict_throughput(res = list(resource_allocation.values()),
                                                    input_throughput= input_throughput,
                                                    differentiable=0,
                                                    res_normalized=False)
print(output_throughput)

<div style="display: inline-block;">
    <h2 style="font-family: 'Courier New', Courier, monospace; font-size: 1.5em; color: #f0f6fc; background-color: #2d333b; border-left: 4px solid #3F51B5; padding: 8px 15px; border-radius: 4px; margin-top: 20px; box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);">
        3b. Validating the algorithm
    </h2>
</div>


In [ ]:
input_throughput = 20
packet_loss = None
output_throughput = 20

if packet_loss is not None:
    output_throughput = input_throughput * (1 - packet_loss*0.01)

resource_allocation_norm, qos, _, _, _ = microopt(slice_model, input_throughput, output_throughput, verbose=1)
resource_allocation_norm = resource_allocation_norm[0]
resource_allocation = [upf_data_gen.denormalize(resource_allocation_norm[0], feature_type='input', feature='res'), 
                              ovs_data_gen.denormalize(resource_allocation_norm[1], feature_type='input', feature='res'), 
                              ran_data_gen.denormalize(resource_allocation_norm[2], feature_type='input', feature='res')]
print(resource_allocation)
print(qos)


In [ ]:
import numpy as np
import ipywidgets as widgets
from plotly import graph_objs as go
from IPython.display import display

# Define sliders for resource allocations and input throughput
upf_slider = widgets.IntSlider(
    value=350, min=200, max=500, step=5,
    description='UPF CPU (millicores):',
    continuous_update=True,
    style={'description_width': '200px'},
    layout=widgets.Layout(width='50%', padding='10px 0px'))

ovs_slider = widgets.IntSlider(
    value=30, min=10, max=50, step=0.1,
    description='OVS Throughput (Mbps):',
    continuous_update=True,
    style={'description_width': '200px'},
    layout=widgets.Layout(width='50%', padding='10px 0px')
)

ran_slider = widgets.IntSlider(
    value=1125, min=250, max=2000, step=1,
    description='RAN CPU (millicores):',
    continuous_update=True,
    style={'description_width': '200px'},
    layout=widgets.Layout(width='50%', padding='10px 0px')
)

input_throughput_slider = widgets.FloatSlider(
    value=27.5, min=5, max=50, step=0.1,
    description='Input Throughput (Mbps):',
    continuous_update=True,
    style={'description_width': '200px'},
    layout=widgets.Layout(width='50%', padding='10px 0px'))

# Metric labels and initial display values
metric_labels = ["Input Throughput", "UPF CPU", "OVS Throughput", "RAN CPU", "Output Throughput", "Packet Loss"]
output_values = {label: widgets.Label("No data") for label in ["Output Throughput", "Packet Loss"]}

# Initialize Plotly figure for metrics visualization
fig = go.FigureWidget(
    data=[go.Bar(x=metric_labels, y=[0, 0, 0, 0, 0, 0], marker_color='skyblue')],
    layout=go.Layout(
        title="Metrics Visualization",
        xaxis=dict(title="Metrics"),
        yaxis=dict(title="Normalized Value", range=[0, 1])
    )
)

# Function to update plot based on slider values
def update_plot(input_throughput, upf_value, ovs_value, ran_value):
    # Predict output throughput and calculate packet loss
    output_throughput = slice_model.predict_throughput(
        res=[upf_value, ovs_value, ran_value],
        input_throughput=input_throughput,
        res_normalized=False
    )
    packet_loss = max(0, (input_throughput - output_throughput) / input_throughput)

    # Normalize values for the bar chart
    normalized_values = [
        input_throughput / 50,
        upf_value / 500,
        ovs_value / 50,
        ran_value / 2000,
        output_throughput / 50,
        packet_loss
    ]
    
    # Update widgets for output values
    output_values["Output Throughput"].value = f"{output_throughput:.2f} Mbps"
    output_values["Packet Loss"].value = f"{packet_loss:.2%}"
    
    # Update Plotly chart
    fig.data[0].y = normalized_values
    fig.update_layout(title="Metrics Visualization")

# Event handler for slider changes
def on_slider_change(change):
    update_plot(
        input_throughput_slider.value,
        upf_slider.value,
        ovs_slider.value,
        ran_slider.value
    )

# Link sliders to the update function
upf_slider.observe(on_slider_change, names='value')
ovs_slider.observe(on_slider_change, names='value')
ran_slider.observe(on_slider_change, names='value')
input_throughput_slider.observe(on_slider_change, names='value')

# Display widgets and initial plot setup
display(upf_slider, ovs_slider, ran_slider, input_throughput_slider)
for label, widget in output_values.items():
    display(widgets.HBox([widgets.Label(f"{label}:", layout=widgets.Layout(width='150px')), widget]))
display(fig)

# Run initial update with default slider values
update_plot(input_throughput_slider.value, upf_slider.value, ovs_slider.value, ran_slider.value)

<div style="font-family: 'Courier New', Courier, monospace; font-size: 1.8em; color: #f6f8fa; background-color: #24292e; border: 2px solid #444d56; border-radius: 6px; padding: 10px 20px; margin-top: 20px; box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);">
    <span style="font-weight: 600; font-size: 1.1em; letter-spacing: 0.5px; color: #f0f6fc;">4. Conclusion</span>
</div>
